# Python for (open) Neuroscience

_Lecture 0.4_ - Classes and objects

Luigi Petrucco

Jean-Charles Mariani

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec/blob/main/lectures/Lecture0.4_Classes-objects.ipynb)

## Classes and objects

Deep down the rabbit (Python) hole...

## Object-oriented programming (OOP)

OOP is a programming paradigm based on the concept of <span style="color:indianred">objects</span>, which _bind together data and code_.

**Disclaimer 0:** 

OOP can be confusing at first. You're entering the realm of True Programming Problems! Some concepts might feel a bit metaphysical.

**Disclaimer 1:**

This lecture is aimed more at understanding/reading existing code than at implementation. You won't probably write a lot of classes right now, but the concepts are essential in Python!

**Disclaimer 1:**

OOP is not intrinsically good (or bad). It can be a very good solution for some coding subtasks, and in Python can be effectively combined with functional programming!

Let's start with an example...

Imagine the following scenario:

 - You have an experiment, containing data, info on subjects, details on experimental stimulus, etc.
 - You want to implement some functions that operate on this dataset
 

First option: one entity -> many variables, many functions:

In [ ]:
# A bunch of variables referring to the same experiment:
def load(data_path):
    ...


sampling_frequency, imaging_data, stimulus_data = load(data_path)


def resample(imaging_data, sampling_frequency, new_frequency=...):
    ...


def crop_on_stimulus(imaging_data, stimulus_data, sampling_frequency, padding=...):
    ...


resample(imaging_data, sampling_frequency, new_frequency=...)
crop_on_stimulus(imaging_data, stimulus_data, sampling_frequency, padding=...)

But: many variables around, not nested in a single entity

Second option: one entity -> one variable, many functions:

In [ ]:
# A bunch of variables referring to the same experiment:
data_dictionary = load(data_path)


def resample(data_dictionary, new_frequency=...):
    data_dictionary["imaging_data"]
    data_dictionary["sampling_frequency"]
    ...


def crop_on_stimulus(data_dictionary, padding=...):
    data_dictionary["imaging_data"]
    data_dictionary["stimulus_data"]
    data_dictionary["sampling_frequency"]
    ...


resample(data_dictionary, new_frequency=...)
crop_on_stimulus(data_dictionary, padding=...)
...

But: the definition of the dictionary and the usage of keywords linger around in many places

It would be very useful to represent together data that refer to some entity, and operations that we can do on them!

## \*Enter objects\*

Objects are entities that keep together:

 - <span style="color:indianred">Attributes</span>: similar to variables, kept together in an object as they would be in a dictionary

 - <span style="color:indianred">Methods</span>: similar to functions, but operating on all the attributes of an object (potentially taking more inputs)

Third option: one entity, one class object with attributes and methods:

In [ ]:
class ExperimentData():
    ...
    def __init__(self, data_path):
        self.imaging_data = ...
        self.stimulus_data = ...
        
    def resample(self, new_frequency=):
        ...
    
    def crop_on_stimulus(self, padding=):
        ...
        
exp_data = ExperimentData(data_path)

exp_data.imaging_data  # access an object's attribute
exp_data.stimulus_data  # access an object's attribute

exp_data.resample(new_frequency=...)  # call an object's method
exp_data.crop_on_stimulus(padding=...)  # call an object's method


Advantages of OOP:

 - represent together data and procedures operating on them
 - flexible data interface ("A dictionary on steroids")
 - nicely define what to expose and what to keep private (abstraction - more on this at the end)

### Platonic coding

To use objects, we first have to define classes.

- classes are the abstract definition of categories from which we create specific instances

- individual, separate objects are then created from a class

- classes are to objects what Ideas are to real world objects for Plato (if that can help at all)

An example for the more pragmatically minded:

In [ ]:
# define void Horse class and make instances:
class Horse:
    pass


my_horse = Horse()
your_horse = Horse()

## Anatomy of a class

What do we need to define a class?

In [ ]:
class TestClass:
    """Showcase the definition of a class."""

    def __init__(self):
        self.an_attribute = 1  # attributes defined IN THE INIT!
        self.another_attibute = {"a": 2}

    def a_method(self):
        ...

  - a <span style="color:indianred">name</span> that describes it. To <span style="color:indianred">istantiate</span> (_i.e_ to make) objects, we use name followed by `()`. CamelCase formatting
  - <span style="color:indianred">attributes</span>, variables that are attributed to the object (or to the class - see later)
  -  <span style="color:indianred">methods</span>, functions that operate on the attributes (and other arguments)
  - (optional but strongly recommended): a <span style="color:indianred">docstring</span> (a documentation string)

Let's make one!

In [138]:
# put some content in a Horse class:


class Horse:
    def __init__(self, name):
        self.name = name
        self.color = "black"

    def run(self):
        print("I'm running very fast")


my_horse = Horse("John")
my_horse.name

'John'

### Inspect objects with `dir()`

We can check out attributes and methods of a class with the base Python function `dir`

In [139]:
my_horse = Horse("Bob")

dir(my_horse)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'color',
 'name',
 'run']

### Look at attributes

To get an attribute, we can use the `obj.attribute` syntax...

In [140]:
all_attributes = dir(my_horse)

...or the `getattr(obj, attribute)` function:

In [141]:
attribute_that_we_want = all_attributes[-2]
getattr(my_horse, attribute_that_we_want)

'Bob'

## Call methods

A method is just a callable attribute! We can access it in the same ways as we do with attributes.

In [142]:
my_horse.run()

I'm running very fast


Or, after using `getattr ()`:

In [143]:
the_run_method = getattr(my_horse, "run")
the_run_method()

I'm running very fast


### `callable()`

We can verify what is callable and what is not using the `callable()` function:

In [147]:
the_run_method = getattr(my_horse, "run")
print("run method: ", callable(the_run_method))

the_name_attribute = getattr(my_horse, "name")
print("name attribute: ", callable(the_name_attribute))

run method:  True
name attribute:  False


You should get along with objects, because in Python objects are quite common!

Strings are objects...

In [40]:
a_string = "some text"
a_string.lower()

'some text'

...but also all other variables are objects...

In [148]:
an_integer = 1
print(dir(an_integer))
an_integer.imag

['__abs__', '__add__', '__and__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floor__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__le__', '__lshift__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift__', '__rmod__', '__rmul__', '__ror__', '__round__', '__rpow__', '__rrshift__', '__rshift__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__trunc__', '__xor__', 'as_integer_ratio', 'bit_length', 'conjugate', 'denominator', 'from_bytes', 'imag', 'numerator', 'real', 'to_bytes']


0

...modules that you import are objects...

In [54]:
import random

random.randint  # functions from imported modules are used as methods of a module object!

<bound method Random.randint of <random.Random object at 0x126819e10>>

...functions are objects... 😦

In [149]:
def an_empty_function():
    """A doctstring."""
    pass


an_empty_function.__doc__  # Docstring of the function referenced as an attribute of the function object

'A doctstring.'

...classes are objects! 🤯

In [153]:
class AClass:
    """Test class."""

    pass


print(dir(AClass))
print(
    AClass.__doc__
)  # Docstring of the class referenced as an attribute of the class object

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']
Test class.


(Practical 0.4.0)

## Defining new classes

We can create our own new custom classes! Sweet, isn't it?

### The `.__init__()` method

Each time we istantiate a new object from a class, the `.__init__()` method is called:

In [66]:
class Horse:
    def __init__(self, name, color):
        print("I'm creating a horse object")
        self.name = name
        self.color = color


my_horse = Horse("Bob", "white")

I'm creating a horse object


We can pass arguments to the `.__init__()` method as we would do for any other function!

In [70]:
# Define function with default values / pass by position or keyword


class Horse:
    def __init__(self, name, color="brown"):
        print("I'm creating a horse object")
        self.name = name
        self.color = color


my_horse = Horse(name="Bob", color="pink")
my_horse.color

I'm creating a horse object


'pink'

### Why all those underscores?

The `__xxxxx__()` name of a method distinguishes it graphically as a <span style="color:indianred">default method</span>.

There is a number of others!

Other examples:
 - `__repr__()` and `__str__()` change how an object is shown/printed:

In [74]:
class Horse:
    def __str__(self):
        return "A string that I choose"

    def __repr__(self):
        return "Another string"


Horse()

Another string

- `__eq__(self, other)` changes how the object is compared with an `other` object:

In [82]:
class ShopList:
    def __init__(self, items):
        self.items = items

    def __eq__(self, the_other_element_in_the_comparison):
        return set(self.items) == set(the_other_element_in_the_comparison.items)


a_shop_list = ShopList(["bread", "salt"])
another_shop_list = ShopList(["salt", "bread"])
a_third_shop_list = ShopList(["salt", "bread", "coffee"])

a_shop_list == another_shop_list

True

- `__getitem__(self, item)` allow us to index the object with square brackets, making it <span style="color:indianred">subscriptable</span> (as lists are!)

In [95]:
class MATLABList:
    def __init__(self, items):
        self.items = items

    def __getitem__(self, item_idx):
        return self.items[item_idx - 1]


shop_list = MATLABList(["bread", "salt"])
a = shop_list[2]
print(a)

salt


Working with those special methods can be very powerful! There is almost nothing in core Python that we cannot use/emulate/modify in the behavior of our new classes!

### Add custom methods to the class

We can create our custom methods in the class:

In [96]:
# a running horse:
class Horse:
    def __init__(self, velocity=10):
        self.position = 0
        self.velocity = 10

    def run_for_n_hours(self, hours):
        self.position += hours * self.velocity


a_horse = Horse()
print(a_horse.position)
a_horse.run_for_n_hours(3)
print(a_horse.position)

0
30


Apart from taking self as the first argument, a method is then just a simple function! It can have positional or keyword values, default values, should be annotated in the same way, etc.

In [111]:
# a running horse:
class Horse:
    def __init__(self, velocity=10):
        self.position = 0
        self.velocity = 10

    def run_for_n_hours(self, hours):
        self.position += hours * self.velocity

In [113]:
a_horse = Horse()
print(a_horse.position)
a_horse.run_for_n_hours(5)
print(a_horse.position)

0
50


### A class and its `self`

    ⚠️ Confusion legit here! ⚠️
    
`self.` is the way in which the specific object instance is referenced to inside the class. this is why we are passing it always to every method - so that it can operate on the value of the object

In [116]:
class IntrospectiveClass:
    def introspect(self):
        print(f"Who am I? I seem to be: {self}")


an_instance = IntrospectiveClass()
an_instance.introspect()

another_instance = IntrospectiveClass()
another_instance.introspect()

Who am I? I seem to be: <__main__.IntrospectiveClass object at 0x11362c970>
Who am I? I seem to be: <__main__.IntrospectiveClass object at 0x11362c3a0>


The source of a funny bug, the lack of a self:

In [117]:
introspective_obj_wrong = (
    IntrospectiveClass  # here we forgot to use the round brackets!
)
introspective_obj_wrong.introspect()  # therefore, this has no self and will crash

TypeError: introspect() missing 1 required positional argument: 'self'

(Practical 0.4.1)

## Inheritance

An important feature of objects is the concept of <span style="color:indianred">inheritance</span>

After we have defined a general class, we can build on it in <span style="color:indianred">children classes</span> or <span style="color:indianred">subclasses</span> to define subtypologies of that entity 

An example:

In [119]:
class Animal:  # Main class or parent class
    def __init__(self, name):
        self.name = name

    def greetings(self):
        print(f"Hi there! I'm {self.name}")


an = Animal("Bob")
an.greetings()

Hi there! I'm Bob


In [123]:
class Horse(Animal):  # Subclass Animal and add a method
    def do_the_horse(self):
        print("I am an horse!")

In [125]:
# This will have all the attributes and methods of the parent

my_horse = Horse(name="John")
my_horse.greetings()

Hi there! I'm John


We can overwrite attributes and methods of the parent class:

In [126]:
class Horse(Animal):  # Subclass Animal
    def greetings(self):
        # this will overwrite the parent's method:
        print(f"I am {self.name} the horse!")


my_horse = Horse(name="John")
my_horse.greetings()

I am John the horse!


Or we can mix the parent methods with some new functionality. To invoke the methods of the parent class we use the `super()` syntax:

In [129]:
class Horse(Animal):  # Subclass Animal and add new methods
    def greetings(self):
        super().greetings()
        print("I am an horse!")


my_horse = Horse(name="John")
my_horse.greetings()  # this will print both the parent and the new message now:

Hi there! I'm John
I am an horse!


For example, we can also redesign the `__init__` method to take more arguments, but we have to make sure we pass all the arguments the parent class needs!

In [132]:
class Horse(Animal):  # Subclass Animal
    # now take color as a possible input:
    def __init__(self, name, color):
        super().__init__(name=name)  # pass name to the parent
        self.color = color  # additional attribute


my_horse = Horse("Jaimie", "Blue")
print(f"Color: {my_horse.color}, name: {my_horse.name}")

Color: Blue, name: Jaimie


(Practical 0.4.2)

### The `*args` `**kwargs` syntax

    ⚠️ Confusion legit here! ⚠️
    
With the `*args`, `**kwargs` trick we can pass to the parent class method all the arguments passed to the `__init__()` method by position or by keyword without having to specify them.

Brace yourself! Expect non-trivial behavior if you venture into this syntax usage! 

The `*args` trick works as a "catch-all" placeholder that gets all positional arguments that are not explicitely defined in a single tuple:

In [ ]:
def arbitrary_inputs_function(first_argument, *args):
    print(f"Captured argument: {first_argument}")
    print(type(args), args)


arbitrary_inputs_function(1, 2, 3)

The `**kwargs` works in the same way: a "catch-all" placeholder that gets all keyword arguments in a single dictionary:

In [ ]:
def arbitrary_inputs_function(first_argument, *args, **kwargs):
    print(f"Captured argument: {first_argument}")
    print(f"args ({type(args)}): {args}")
    print(f"kwargs ({type(kwargs)}): {kwargs}")


arbitrary_inputs_function(1, 2, 3, random_kwarg=5)

Using them in a class:

In [ ]:
class Horse(Animal):  # Subclass Animal
    def __init__(self, *args, color="black"):
        super().__init__(*args)

        self.color = color

    def greetings(self):
        print(self.color, "name", self.name)


h = Horse("bob", color="white")
h.greetings()

### Class attributes

We can specify attributes at the level of the class - _i.e._ out of the init, and without referencing the self

In [ ]:
class ClassWithClassAttributes:
    class_attribute = []

    def __init__(self):
        self.object_attribute = []


an_obj = ClassWithClassAttributes()
another_obj = ClassWithClassAttributes()

another_obj.object_attribute.append(1)  # this will modify only the object

print(
    f"Object attributes after changing one: {an_obj.object_attribute}, {another_obj.object_attribute} "
)

another_obj.class_attribute.append(
    1
)  # this will modify the list in all instances of the class
print(
    f"Class attributes after changing one: {an_obj.class_attribute},  {another_obj.class_attribute}"
)

### Keeping class business private

There is a convention for classes to define attributes and methods that are only accessed internally with a leading `_`. This is just a convention, and will not change the behavior of such attributes and methods!

In [ ]:
class TestUnderscores:
    def __init__(self):
        self._private_var = 1

    def _update_private_var(self):
        self._private_var


TestUnderscores()._private_var

With this syntax, you know that if you are accessing that value from the outside you are using the class out of the intended ways.

### Enforcing privacy

If you want something to be really not reachable from the outside, you can use double leading `__` (called dunder).

In [ ]:
class TestUnderscores:
    def __init__(self):
        self.__private_var = 0

    def update_private_var(self):
        # this is called from inside the function so it is fine!
        self.__private_var = 1


TestUnderscores().update_private_var()

But, this is not really private! Python is just hiding it from us...If we use the dir() function we can se what is happening!

In [ ]:
dir(TestUnderscores())